<a href="https://colab.research.google.com/github/kevgam/CAS_IE_Information_Retrieval/blob/main/IR_TEST_ipywidgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Importieren der Bibliotheken

import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import nltk as nlt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# NLTK-Resourcen herunterladen
nlt.download('punkt')
nlt.download('stopwords')


AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [ ]:
# Google Drive verbinden

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Lyrics laden
df = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_bereinigt.csv')


In [ ]:
# Lyrics-Suche

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF-Vektorisierung der Lyrics
vectorizer = TfidfVectorizer(stop_words='english')

# Vektorisierung der bereinigten Lyrics
tfidf_matrix = vectorizer.fit_transform(df['clean_lyrics'])

# Widget für die Lyrics-Eingabe
lyrics_input = widgets.Textarea(
    value='',
    placeholder='Füge deine Stimmung ein... ',
    description='Lyrics:',
    disabled=False
)

# Funktion, um die besten Treffer basierend auf den eingegebenen Lyrics zu finden
def search_similar_songs(button):
    query = lyrics_input.value
    if query:
        query_vec = vectorizer.transform([preprocess_text(query)])  # Vorverarbeitung der Eingabe
        cosine_sim = cosine_similarity(query_vec, tfidf_matrix)  # Berechnung der Kosinus-Ähnlichkeit
        top_n = 5  # Top 5 ähnliche Songs
        top_indices = cosine_sim.argsort()[0][-top_n:][::-1]  # Indizes der besten Übereinstimmungen
        results = df.iloc[top_indices][['Songname', 'clean_lyrics', 'Thema']]  # Ergebnisse filtern
        print("Ähnliche Songs:")
        print(results)
    else:
        print("Bitte füge Lyrics ein.")

# Button für die Lyrics-Suche
search_button = widgets.Button(description="Ähnliche Songs finden")
search_button.on_click(search_similar_songs)

# Anzeige der Widgets
display(lyrics_input, search_button)


In [ ]:
# Stimmungs-Suche:


 Widget für die Stimmungswahl
mood_input = widgets.Dropdown(
    options=['Liebe', 'Verlust', 'Freude', 'Widerstand', 'Trauer'],
    value='Liebe',
    description='Stimmung:',
    disabled=False
)

# Funktion, um Songs nach Stimmung zu filtern
def search_by_mood(button):
    mood = mood_input.value
    filtered_songs = df[df['Thema'] == mood][['Songname', 'clean_lyrics']]
    if not filtered_songs.empty:
        print(f"Songs mit dem Thema {mood}:")
        print(filtered_songs)
    else:
        print(f"Keine Songs mit dem Thema {mood} gefunden.")

# Button für die Stimmungs-Suche
mood_search_button = widgets.Button(description="Songs nach Stimmung suchen")
mood_search_button.on_click(search_by_mood)

# Anzeige der Widgets
display(mood_input, mood_search_button)
